# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Carry out some basic tests on the CSV file data - NOT IN USE ONCE ETL IS LIVE

In [13]:
#test_df = pd.read_csv('event_datafile_new.csv')
#test_df.head(3) 

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88


In [14]:
# test 1 - check if "itemInSession" & "sessionId" combined are unique - if unique, answer should be 0. This means can be used as Primary Key.
#len(test_df[test_df.duplicated(subset=['itemInSession','sessionId'], keep=False)]) 

0

In [15]:
# clear test data 
#del test_df 

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
#drop keyspace if exists
try:
    session.execute("DROP KEYSPACE IF EXISTS sparkify")   
except Exception as e:
    print(e) 

# create new keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1}"""
) 
    
except Exception as e:
    print(e) 

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print("Setting keyspace has failed") 
    print(e) 

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [10]:
# drop table if exists - then create table fresh 
query1_a = "DROP TABLE IF EXISTS music_app_hst_1" 
query1_b = """CREATE TABLE IF NOT EXISTS music_app_hst_1
              (itemInSession text, sessionId text, artist text, song text, length text, PRIMARY KEY (itemInSession, sessionId)) 
"""

# process drop table statement
try:
    session.execute(query1_a) 
except Exception as e:
    print("Query failed")
    print(e) 
    
# process create table statement 
try:
    session.execute(query1_b) 
except Exception as e:
    print("Query failed")
    print(e) 

In [11]:
# collect data from CSV file and read each line into the table created in the step above 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_hst_1 (itemInSession, sessionId, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"

        session.execute(query, (line[3], line[8], line[0], line[9], line[5])) 

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
test_query1 = "select artist, song, length from music_app_hst_1 WHERE itemInSession='4' AND sessionId='338'"
try:
    rows = session.execute(test_query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length) 

Faithless Music Matters (Mark Knight Dub) 495.3073


In [18]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# drop table if exists - then create table fresh 
query2_a = "DROP TABLE IF EXISTS music_app_hst_2" 
query2_b = """CREATE TABLE IF NOT EXISTS music_app_hst_2
              (userId text, sessionId text, itemInSession text, artist text, song text, firstName text, lastName text, PRIMARY KEY (userId, sessionId, itemInSession)) 
"""

# process drop table statement
try:
    session.execute(query2_a) 
except Exception as e:
    print("Query failed")
    print(e) 
    
# process create table statement 
try:
    session.execute(query2_b) 
except Exception as e:
    print("Query failed")
    print(e)             

In [19]:
# collect data from CSV file and read each line into the table created in the step above 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_hst_2 (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"

        session.execute(query, (line[10], line[8], line[3], line[0], line[9], line[1], line[4])) 

#### Do a SELECT to verify that the data have been inserted into each table

In [21]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
test_query2 = "select artist, song, firstName, LastName from music_app_hst_2 WHERE userId='10' AND sessionId='182'"
try:
    rows = session.execute(test_query2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname) 

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [22]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# drop table if exists - then create table fresh 
query3_a = "DROP TABLE IF EXISTS music_app_hst_3" 
query3_b = """CREATE TABLE IF NOT EXISTS music_app_hst_3
              (song text, userId text, firstName text, lastName text, PRIMARY KEY (song, userId)) 
"""

# process drop table statement
try:
    session.execute(query3_a) 
except Exception as e:
    print("Query failed")
    print(e) 
    
# process create table statement 
try:
    session.execute(query3_b) 
except Exception as e:
    print("Query failed")
    print(e)        

In [23]:
# collect data from CSV file and read each line into the table created in the step above 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_app_hst_3 (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"

        session.execute(query, (line[9], line[10], line[1], line[4])) 

In [25]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
test_query3 = "select firstName, LastName from music_app_hst_3 WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(test_query3)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname) 

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [ ]:
## TO-DO: Drop the table before closing out the sessions

In [26]:
# process drop table statement
drop1 = "DROP TABLE IF EXISTS music_app_hst_1"
drop2 = "DROP TABLE IF EXISTS music_app_hst_2"
drop3 = "DROP TABLE IF EXISTS music_app_hst_3"

try:
    session.execute(drop1) 
except Exception as e:
    print("drop1 failed")
    print(e) 
    
try:
    session.execute(drop2) 
except Exception as e:
    print("drop2 failed")
    print(e) 
    
try:
    session.execute(drop3) 
except Exception as e:
    print("drop3 failed")
    print(e) 

### Close the session and cluster connection¶

In [27]:
session.shutdown()
cluster.shutdown()